# 07-01 : Basic Trino Query

Test using Trino without MLRun at this stage.

In [1]:
from contextlib import closing

import pandas as pd
import trino
from jinja2 import Environment, FileSystemLoader

## 1. Configuration

In [2]:
config = {
    "TRINO_HOST": "dragon.lan",
    "TRINO_PORT": "9191",
    "TRINO_USER": "johnny",
}

## 2. Establish the connection to Trino

In [3]:
conn = trino.dbapi.connect(
    host=config["TRINO_HOST"],
    port=config["TRINO_PORT"],
    user=config["TRINO_USER"],
    schema="lakehouse",
    catalog="iceberg",
)

## 3. Test Query Execution

### 3.1 Basic Query

In [4]:
query = "SELECT * FROM lakehouse.taxi_trips LIMIT 10"

In [5]:
# Execute the query and fetch the data into a pandas DataFrame
with closing(conn.cursor()) as cur:
    cur.execute(query)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=columns)

df.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,0fca59218b11688279d795c03c4d16f851f13fa0,e2c349c7cbb608d552aa0b5814031943f13641ef9e50d8...,2023-01-01 00:00:00,2023-01-01 00:15:00,1037,4.82,None,None,6.0,32.0,...,0.0,19.50,Credit Card,Taxicab Insurance Agency Llc,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841)
1,1e539d6e7501164c6b76b761c3152c235e206d59,4ab7a7510c1ebcc9b2e3eaa7bdd6508dbea34da7986aca...,2023-01-01 00:00:00,2023-01-01 00:15:00,1341,16.63,None,None,76.0,8.0,...,6.0,53.00,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
2,79d2eff4b65d9bb126c12ded2f76c67f07c328ab,f6e15e1e93c7648c1ffb637834e78905aca64ba09e2f46...,2023-01-11 14:15:00,2023-01-11 14:30:00,1167,7.99,None,None,NaN,NaN,...,0.0,23.00,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None
3,79d3b3527ba03b3b487a220d3592fb3c92d78aaf,121e6b6064d993f2f1bc475ffd6392987c431f18e99c30...,2023-01-23 21:00:00,2023-01-23 21:15:00,240,0.70,None,None,NaN,NaN,...,0.0,7.25,Credit Card,Top Cab Affiliation,NaN,NaN,None,NaN,NaN,None
4,79f38d1640cc5636fca1aa533474879e5f5d8b95,92aa59b10af01f1beaa1a0ad435c030f2efc4107fbaeb0...,2023-01-08 15:15:00,2023-01-08 15:30:00,1171,14.55,None,None,NaN,NaN,...,25.5,75.60,Credit Card,City Service,NaN,NaN,None,NaN,NaN,None


### 3.2 AIB Ingest Query Parts

#### 3.2.1 Filter

In [6]:
# the query template
query_template = """
    SELECT
        *
    FROM
        "{{ catalog }}"."{{ schema }}"."{{ source_table }}"
    WHERE
        CAST({{ filter_column }} AS DATE) BETWEEN
            (CAST('{{ filter_start_value }}' AS DATE) - INTERVAL '3' MONTH) AND
            (CAST('{{ filter_start_value }}' AS DATE) - INTERVAL '2' MONTH)
"""

# the template parameters
params = {
    "catalog": "iceberg",
    "schema": "lakehouse",
    "source_table": "taxi_trips",
    "filter_column": "trip_start_timestamp",
    "filter_start_value": "2023-05-01"
}

In [7]:
# Create a Jinja2 environment and load the template from the string
template = Environment().from_string(query_template)

# Render the template by passing the parameters
rendered_query = template.render(params)

# Print the rendered query to see the final result
print("--- Rendered SQL Query ---")
print(rendered_query)
print("--------------------------")

--- Rendered SQL Query ---

    SELECT
        *
    FROM
        "iceberg"."lakehouse"."taxi_trips"
    WHERE
        CAST(trip_start_timestamp AS DATE) BETWEEN
            (CAST('2023-05-01' AS DATE) - INTERVAL '3' MONTH) AND
            (CAST('2023-05-01' AS DATE) - INTERVAL '2' MONTH)
--------------------------


In [8]:
with closing(conn.cursor()) as cur:
    cur.execute(rendered_query)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=columns)
    
# display the result
print(f"{df.shape[0]} rows returned")
display(df.head())

465977 rows returned


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,2b5ead82644ecb649f7e73631b7ad9f278366b30,86b07dc8beb256766f2025da2b99b77345074852eabbdf...,2023-02-22 16:30:00,2023-02-22 17:00:00,1865.0,10.61,NaN,NaN,NaN,NaN,...,4.0,42.50,Credit Card,Sun Taxi,NaN,NaN,None,NaN,NaN,None
1,2e3e1c4a070e854d427869738adcfd094036c2e1,65f42dfde0c75a7feb7133d394bff7140ae249ca450a01...,2023-02-23 15:00:00,2023-02-23 16:00:00,3000.0,18.10,NaN,NaN,NaN,NaN,...,0.0,57.30,Credit Card,Globe Taxi,NaN,NaN,None,NaN,NaN,None
2,2d28666dd3f87a6e2ef719382b3bec7caa0df14a,3d0f28391371d13655fe6916bb5304225945545eccc922...,2023-02-03 08:45:00,2023-02-03 08:45:00,60.0,0.00,NaN,NaN,NaN,NaN,...,49.5,52.75,Cash,Choice Taxi Association,NaN,NaN,None,NaN,NaN,None
3,310dcde3b3ec2ef1a1c343f41f477ef834973e6a,ee6216cc0f10e2d9b4a4e875fa910a413dde12495038a5...,2023-02-06 13:15:00,2023-02-06 13:15:00,4.0,0.00,NaN,NaN,NaN,NaN,...,0.0,102.60,Credit Card,Star North Taxi Management Llc,NaN,NaN,None,NaN,NaN,None
4,2c28d3af85ee243ed0f4fa10d60345bc2859dba8,ff60dabe17243a25435dcaf430a8b31615374bfc6be841...,2023-02-05 19:30:00,2023-02-05 19:30:00,414.0,0.00,NaN,NaN,NaN,NaN,...,0.0,12.39,Mobile,Flash Cab,NaN,NaN,None,NaN,NaN,None


#### 3.2.2 Full Query

In [12]:
# --- 1. Define your full query template ---
# This is the complete Trino SQL you provided.
query_template = """
WITH filtered_data AS (
    SELECT
        *
    FROM
        "{{ catalog }}"."{{ schema }}"."{{ source_table }}"
    WHERE
        -- Trino uses CAST(... AS DATE) and the standard subtraction operator for intervals
        CAST({{ filter_column }} AS DATE) BETWEEN
            (CAST('{{ filter_start_value }}' AS DATE) - INTERVAL '3' MONTH) AND
            (CAST('{{ filter_start_value }}' AS DATE) - INTERVAL '2' MONTH)
)

, mean_time AS (
    SELECT
        -- Trino's equivalent of INT64 is BIGINT
        CAST(AVG(trip_seconds) AS BIGINT) AS avg_trip_seconds
    FROM
        filtered_data
)

SELECT
    -- Function names for date/time extraction are slightly different
    CAST(day_of_week(trip_start_timestamp) AS DOUBLE) AS dayofweek, -- Note: Sunday=7 in Trino
    CAST(hour(trip_start_timestamp) AS DOUBLE) AS hourofday,
    
    -- Geospatial functions use a different syntax
    ST_Distance(
        to_spherical_geography(ST_Point(pickup_longitude, pickup_latitude)),
        to_spherical_geography(ST_Point(dropoff_longitude, dropoff_latitude))
    ) AS trip_distance,
    
    trip_miles,
    
    -- Trino's equivalent of FLOAT64 is DOUBLE
    CAST(
        CASE
            WHEN trip_seconds IS NULL THEN m.avg_trip_seconds
            WHEN trip_seconds <= 0 THEN m.avg_trip_seconds
            ELSE trip_seconds
        END AS DOUBLE
    ) AS trip_seconds,
    
    payment_type,
    company,
    
    -- Use double quotes for identifiers if needed, not backticks
    (fare + tips + tolls + extras) AS "{{ target_column }}"
FROM
    filtered_data AS t
-- Explicit CROSS JOIN is clearer than a comma
CROSS JOIN mean_time AS m
WHERE
    trip_miles > 0 AND fare > 0 AND fare < 1500
    -- The Jinja templating part does not need to change at all
    {% for field in ["fare", "trip_start_timestamp", "pickup_longitude",
                "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "payment_type", "company"] %}
        AND "{{ field }}" IS NOT NULL
    {% endfor %}
"""

# --- 2. Create a dictionary with all required parameters ---
# We just need to add the 'target_column' to our previous dictionary.
params = {
    "catalog": "iceberg",
    "schema": "lakehouse",
    "source_table": "taxi_trips",
    "filter_column": "trip_start_timestamp",
    "filter_start_value": "2023-05-01",
    "target_column": "total_fare"  # Added the parameter for the final column alias
}

# --- 3. Use Jinja2 to render the final query ---
template = Environment().from_string(query_template)
rendered_query = template.render(params)

# (Crucial for debugging) Print the fully rendered query to see the final result.
# Notice how the {% for ... %} loop has been expanded into multiple AND clauses.
print("--- Rendered SQL Query ---")
print(rendered_query)
print("--------------------------")

# --- 4. Connect and execute the rendered query ---
config = {
    "TRINO_HOST": "dragon.lan",
    "TRINO_PORT": "9191",
    "TRINO_USER": "johnny",
}

conn = trino.dbapi.connect(
    host=config["TRINO_HOST"],
    port=config["TRINO_PORT"],
    user=config["TRINO_USER"],
    schema=params["schema"],
    catalog=params["catalog"],
)

print("Executing query against Trino...")
# Execute the query and fetch the data into a pandas DataFrame
with closing(conn.cursor()) as cur:
    cur.execute(rendered_query)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=columns)

print("Query finished successfully!")
print(f"Retrieved {len(df)} rows.")
display(df.head())

--- Rendered SQL Query ---

WITH filtered_data AS (
    SELECT
        *
    FROM
        "iceberg"."lakehouse"."taxi_trips"
    WHERE
        -- Trino uses CAST(... AS DATE) and the standard subtraction operator for intervals
        CAST(trip_start_timestamp AS DATE) BETWEEN
            (CAST('2023-05-01' AS DATE) - INTERVAL '3' MONTH) AND
            (CAST('2023-05-01' AS DATE) - INTERVAL '2' MONTH)
)

, mean_time AS (
    SELECT
        -- Trino's equivalent of INT64 is BIGINT
        CAST(AVG(trip_seconds) AS BIGINT) AS avg_trip_seconds
    FROM
        filtered_data
)

SELECT
    -- Function names for date/time extraction are slightly different
    CAST(day_of_week(trip_start_timestamp) AS DOUBLE) AS dayofweek, -- Note: Sunday=7 in Trino
    CAST(hour(trip_start_timestamp) AS DOUBLE) AS hourofday,
    
    -- Geospatial functions use a different syntax
    ST_Distance(
        to_spherical_geography(ST_Point(pickup_longitude, pickup_latitude)),
        to_spherical_geography(ST_P

,dayofweek,hourofday,trip_distance,trip_miles,trip_seconds,payment_type,company,total_fare
0,6.0,19.0,0.000000,10.15,72844.0,Cash,Flash Cab,41.50
1,7.0,22.0,0.000000,0.40,1121.0,Cash,Taxi Affiliation Services,4.00
2,3.0,5.0,0.000000,0.39,330.0,Cash,Flash Cab,69.45
3,1.0,12.0,5158.139389,12.48,1098.0,Prcard,Flash Cab,31.50
4,7.0,11.0,3127.308307,0.60,600.0,Unknown,Taxi Affiliation Services,25.50
